<a href="https://colab.research.google.com/github/Kane0068/testrepo/blob/main/Launch_Sites_Locations_Analysis_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Launch Sites Locations Analysis with Folium**


* Fırlatma başarı oranı, yük kütlesi, yörünge tipi vb. gibi birçok faktöre bağlı olabilir. Ayrıca bir fırlatma sahasının konumuna ve yakınlıklarına, yani roket yörüngelerinin başlangıç konumuna da bağlı olabilir. Bir fırlatma sahası inşa etmek için en uygun konumu bulmak kesinlikle birçok faktörü içerir ve umarım mevcut fırlatma sahası konumlarını analiz ederek bazı faktörleri keşfedebiliriz.

* Önceki keşif amaçlı veri analizi laboratuvarlarında, matplotlib ve seaborn kullanarak SpaceX fırlatma veri setini görselleştirdiniz ve fırlatma yeri ile başarı oranları arasında bazı ön korelasyonlar keşfettiniz. Bu laboratuvarda, Folium'u kullanarak daha etkileşimli görsel analizler yapacaksınız.

Bu laboratuvar aşağıdaki görevleri içerir:

* **GÖREV 1:** Tüm fırlatma alanlarını bir harita üzerinde işaretleyin
* **GÖREV 2:** Haritada her site için başarılı/başarısız lansmanları işaretleyin
* **GÖREV 3:** Bir fırlatma sahası ile yakınlıkları arasındaki mesafeleri hesaplayın

Yukarıdaki görevleri tamamladıktan sonra, fırlatma siteleri hakkında bazı coğrafi modeller bulabilmeniz gerekir.


## 1- İMPORTİNG LİBRARİES

In [1]:
!pip3 install folium
!pip3 install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=a0aed0d89652d29ffe98f812bbd6dc462ddb3ac2ecbe21f56ca3c49ce12b0853
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [2]:
import folium
import wget
import pandas as pd

In [3]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

Folium ile ilgili hafızanızı yenilemeniz gerekirse, bu önceki folium laboratuvarını indirebilir ve başvurabilirsiniz:

## Görev 1: Tüm fırlatma alanlarını bir harita üzerinde işaretleyin

Öncelikle, sitenin enlem ve boylam koordinatlarını kullanarak her sitenin konumunu bir haritaya eklemeye çalışalım

* Aşağıdaki spacex_launch_geo.csv adlı veri kümesi, her site için eklenmiş enlem ve boylam ile artırılmış bir veri kümesidir.

In [4]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [5]:
# Şimdi, her sitenin koordinatlarının ne olduğuna bir göz atabiliriz
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`

spacex_df = spacex_df[['Launch Site','Lat','Long','class']]
launch_sites_df = spacex_df.groupby(['Launch Site'],as_index = False).first()
launch_sites_df = launch_sites_df[['Launch Site','Lat','Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Yukarıdaki koordinatlar, fırlatma sitelerinin nerede olduğu hakkında size herhangi bir sezgisel fikir vermeyen düz sayılardır. Coğrafyada çok iyiyseniz, bu sayıları doğrudan zihninizde yorumlayabilirsiniz. Değilse, bu da iyi. Bu konumları bir haritaya sabitleyerek görselleştirelim.

* İlk olarak, başlangıç merkezi konumu Houston, Teksas'taki NASA Johnson Uzay Merkezi olacak şekilde bir yaprak Haritası nesnesi oluşturmamız gerekiyor.

In [6]:
# Start Location is Nasa Johnson Space center

nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
site_map

Belirli bir koordinatta bir metin etiketi ile vurgulanmış bir daire alanı eklemek için folium.Circle'ı kullanabiliriz. Örneğin,

In [ ]:
# NASA JOHNSON UZAY MERKEZİNİ GÖSTERECEĞİMİZ BİR DAİRE OLUŞTURURYORUZ VE ADINI YAZAN POPUP ETKETİ OLUŞTURUYORUZ
circle = folium.Circle(nasa_coordinate,radius = 1000,color = '#d35400',fill = True).add_child(folium.Popup('Nasa Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon = DivIcon(
        icon_size = (20,20),
        icon_anchor = (0,0),
        html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC'
    )
)


site_map.add_child(circle)
site_map.add_child(marker)

ve Houston şehrinin yakınında küçük sarı bir daire bulmalısınız ve daha büyük bir daire görmek için yakınlaştırabilirsiniz.

Şimdi, launch_sites veri çerçevesindeki her başlatma sitesi için bir daire ekleyelim

In [ ]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

for index, row in launch_sites_df.iterrows():
    coordinates = [row['Lat'], row['Long']]
    folium.Circle(coordinates, radius=1000, color='#000000', fill=True).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)
    folium.map.Marker(coordinates, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'], )).add_to(site_map)
site_map

Artık işaretli alanları yakınlaştırıp/uzaklaştırarak haritayı keşfedebilir ve aşağıdaki soruları yanıtlamaya çalışabilirsiniz:

Tüm fırlatma alanları Ekvator hattına yakın mı?
Tüm fırlatma alanları kıyıya çok yakın mı?
Ayrıca lütfen bulgularınızı açıklamaya çalışın.

##Görev 2: Haritadaki her site için başarılı/başarısız lansmanları işaretleyin

Ardından, her site için başlatma sonuçlarını ekleyerek haritayı geliştirmeye çalışalım ve hangi sitelerin yüksek başarı oranlarına sahip olduğunu görelim. Spacex_df veri çerçevesinin ayrıntılı başlatma kayıtlarına sahip olduğunu ve sınıf sütununun bu başlatmanın başarılı olup olmadığını gösterdiğini hatırlayın.

In [ ]:
spacex_df.tail(10)

Ardından, tüm başlatma kayıtları için işaretçiler oluşturalım. Başlatma başarılıysa (sınıf=1), yeşil işaretçi kullanırız ve başlatma başarısız olursa kırmızı işaretçi kullanırız (sınıf=0)

Bir fırlatmanın yalnızca dört fırlatma sahasından birinde gerçekleştiğini unutmayın; bu, birçok fırlatma kaydının tam olarak aynı koordinata sahip olacağı anlamına gelir. İşaret kümeleri, aynı koordinata sahip birçok işaretçi içeren bir haritayı basitleştirmenin iyi bir yolu olabilir.

In [10]:
# MarkerCluster objesi oluşturarak başlayalım

marker_cluster = MarkerCluster()

> * İşaretçi renklerini sınıf değerine göre depolamak için launch_sites veri çerçevesinde marker_color adlı yeni bir sütun oluşturun



In [11]:
# "sınıf" sütununun değerini kontrol etmek için bir işlev uygulayın
# class=1 ise marker_color değeri yeşil olur
# class=0 ise marker_color değeri kırmızı olur

In [ ]:
def assign_marker_color(launch_outcome):
  if launch_outcome==1:
    return 'green'
  else:
    return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

In [28]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# spacex_df veri çerçevesindeki her satır için
# koordinatıyla birlikte bir Marker nesnesi oluşturun
# ve bu başlatmanın başarılı veya başarısız olduğunu belirtmek için İşaretleyicinin simge özelliğini özelleştirin, 
# örneğin, icon=folium.Icon(color='white', icon_color=row['marker_color']
for i, record in spacex_df.iterrows():
    marker = folium.Marker(location = (spacex_df.iloc[i]['Lat'],spacex_df.iloc[i]['Long']), icon = folium.Icon(color='white', icon_color =spacex_df.iloc[i]['marker_color'])).add_to(marker_cluster)
    marker_cluster.add_child(marker)

site_map

İşaretleyici kümelerindeki renk etiketli işaretleyicilerden, hangi fırlatma sahalarının nispeten yüksek başarı oranlarına sahip olduğunu kolayca belirleyebilmeniz gerekir.

Daha sonra, fırlatma alanlarının yakınlıklarını keşfetmemiz ve analiz etmemiz gerekiyor.

Öncelikle haritadaki bir noktanın üzerine fare ile gelindiğinde koordinat almak için haritaya bir MousePosition ekleyelim. Böylece, haritayı keşfederken, herhangi bir ilgi noktasının (demiryolu gibi) koordinatlarını kolayca bulabilirsiniz

In [29]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Şimdi bir fırlatma sahasına yakınlaşın ve herhangi bir demiryolu, otoyol, sahil şeridi vb. bulup bulamayacağınızı görmek için yakınlığını araştırın. Farenizi bu noktalara getirin ve koordinatlarını (sol üstte gösterilir) fırlatma sahasına olan mesafeye göre işaretleyin.

You can calculate the distance between two points on the map based on their Lat and Long values using the following method:

* Aşağıdaki yöntemi kullanarak harita üzerindeki iki nokta arasındaki mesafeyi Lat ve Long değerlerine göre hesaplayabilirsiniz:

In [30]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

MousePosition kullanarak en yakın kıyı şeridindeki bir noktayı işaretleyin ve kıyı şeridi noktası ile fırlatma sahası arasındaki mesafeyi hesaplayın.

In [34]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
lat = 28.56367 
Lon =-80.57163

distance_coastline = calculate_distance(launch_sites_df.iloc[0]['Lat'],launch_sites_df.iloc[0]['Long'],lat,Lon)
distance_coastline

0.5797581813109574

In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
#Harita üzerinde seçtiğiniz en yakın kıyı şeridi noktasına bir folium.marker oluşturun ve ekleyin
# Simge özelliğini kullanarak kıyı şeridi noktası ile fırlatma sahası arasındaki mesafeyi görüntüleyin 
# örneğin
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )



In [45]:
distance_marker = folium.Marker(
    [lat,Lon],
    icon = DivIcon(
        icon_size = (20,20),
        icon_anchor = (0,0),
        html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )

)

site_map.add_child(distance_marker)

 Bir fırlatma sahası ile seçilen kıyı şeridi noktası arasında bir `PolyLine` çizin


In [44]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)

coordinates = [[lat,Lon],[launch_sites_df.iloc[0]["Lat"], launch_sites_df.iloc[0]["Long"]]]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)


site_map.add_child(lines)

Benzer şekilde, bir fırlatma sahası ile en yakın şehir, demiryolu, otoyol vb. arasında bir çizgi çizebilirsiniz. Harita üzerinde koordinatlarını bulmak için önce MousePosition kullanmanız gerekir

In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

# En yakın şehre, demiryoluna, otoyola vb. uzaklığı gösteren bir işaretleyici oluşturun.
# İşaretleyici ile fırlatma sahası arasında bir çizgi çizin


Yakınlıklara mesafe çizgileri çizdikten sonra aşağıdaki soruları kolayca yanıtlayabilirsiniz:

* Fırlatma sahaları demiryollarına yakın mı?
* Fırlatma sahaları otoyollara yakın mı?
* Fırlatma sahaları kıyı şeridine yakın mı?
* Fırlatma sahaları şehirlerden belirli bir mesafede mi?

Ayrıca lütfen bulgularınızı açıklamaya çalışın.

In [47]:
# Port Saint Lucie Uzaklık Hesabını Yaplım
Lat_PortSLucie = 27.2934
Long_PortSLucie = -80.34963

distance_portslucie = calculate_distance(
    launch_sites_df.iloc[0]['Lat'],
    launch_sites_df.iloc[0]['Long'],
    Lat_PortSLucie,
    Long_PortSLucie)

distance_portslucie

142.9029630091523

In [50]:
# Şİmdi Port Saint Lucie 'ye Marker Yerleştirelim

distance_marker_Lucie = folium.Marker(
    [Lat_PortSLucie,Long_PortSLucie],
    icon =DivIcon(
        icon_size = (20,20),
        icon_anchor = (0,0),
        html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_portslucie),
    )
)



In [53]:
# POrt Saint Lucie 'ye Daire Koyalım

folium.Circle([Lat_PortSLucie,Long_PortSLucie], radius=1000, color='#000000', fill=True).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)

In [54]:

# Çizgiyi Oluşturalım
df_lat =launch_sites_df.iloc[0]['Lat']
df_long =launch_sites_df.iloc[0]['Long']

portLucie_to_df = [[Lat_PortSLucie,Long_PortSLucie],[df_lat,df_long]]
lines_Lucie = folium.PolyLine(
    locations = portLucie_to_df,
    weight = 1
)

site_map.add_child(distance_marker_Lucie)
site_map.add_child(lines_Lucie)